In [1]:
import numpy as np
import glob
import cv2
import os

In [2]:
img = cv2.imread("InputImage/dcJgpal4-5Q.jpg")
cv2.imwrite('InputImage/dcJgpal4-5Q.png', img)
img = cv2.imread("InputImage/dcJgpal4-5Q.png")

In [3]:
height = 900

In [4]:
width = 700

In [5]:
img = cv2.resize(img,(width,height))

In [6]:
files = glob.glob('OutputImage/*')
for f in files:
    os.remove(f)

In [7]:
for j in range(0,height,10):
    for i in range(0,width,10):
        c_im = img[j:j+10,i:i+10]
        c_im = cv2.resize(c_im,(160,160))
        cv2.imwrite("OutputImage/" + str(i) + "_" + str(j) + ".png", c_im)

In [16]:
in_index = {}
in_images = {}
for imagePath in glob.glob("EmojiImages" + "/*.png"):
    # extract the image filename (assumed to be unique) and
    # load the image, updating the images dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    image = cv2.imread(imagePath)
    
    Z = image.reshape((-1,3))

    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 5
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))
    
    in_images[filename] = image
 
    # extract a 3D RGB color histogram from the image,
    # using 8 bins per channel, normalize, and update
    # the index
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8],
        [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist,hist).flatten()
    in_index[filename] = hist
print("OK")

OK


In [17]:
out_index = {}
out_images = {}
for imagePath in glob.glob("OutputImage" + "/*.png"):
    # extract the image filename (assumed to be unique) and
    # load the image, updating the images dictionary
    filename = imagePath[imagePath.rfind("/") + 1:]
    image = cv2.imread(imagePath)
    
    Z = image.reshape((-1,3))

    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 5
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))
    
    out_images[filename] = image
 
    # extract a 3D RGB color histogram from the image,
    # using 8 bins per channel, normalize, and update
    # the index
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8],
        [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist,hist).flatten()
    out_index[filename] = hist
print("OK")

OK


In [18]:
for j in range(0,height,10):
    for i in range(0,width,10):
        val_d = 0.0
        val_k = "name"
        results = {}
        for (k, hist) in in_index.items():
        # compute the distance between the two histograms
        # using the method and update the results dictionary 
            d = cv2.compareHist(out_index[str(i) + "_" + str(j) + ".png"], hist, cv2.HISTCMP_CORREL)
            if d > val_d:
                val_d = d
                val_k = k
#             results[k] = d
#         results = sorted([(v, k) for (k, v) in results.items()], reverse = True)
        new_im = in_images[val_k]
        cv2.imwrite("OUT/" + str(i) + "_" + str(j) + ".png", new_im)

In [19]:
blank_image = np.zeros((height*10,width*10,3), np.uint8)

for j in range(0,height*10,100):
    for i in range(0,width*10,100):
        c_im = cv2.imread("OUT/" + str(i//10) + "_" + str(j//10) + ".png",-1)
        c_im = cv2.resize(c_im,(100,100))
        blank_image[j:j+100,i:i+100] = c_im
blank_image = cv2.resize(blank_image, (width,height))
cv2.imwrite('YES.png', blank_image)

True